<a href="https://www.kaggle.com/code/drkaggle22/toxic-comments-classification-tf-idf-nltk?scriptVersionId=176825803" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_large_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_large_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_large_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_400.npy
/kaggle/input/toxic-comments-transformer-embeddings/train_embs_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_320.npy
/kaggle/input/toxic-comments-transformer-embeddings/test_embs_large_320.npy


In [2]:
df_train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip', index_col='id')

df_test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip', index_col='id')

In [3]:
pd.set_option('display.max_rows', None)

In [4]:
df_train['comment_text'] = df_train['comment_text'].str.lower()
df_test['comment_text'] = df_test['comment_text'].str.lower()

In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153164 entries, 00001cee341fdb12 to ffffce3fb183ee80
Data columns (total 1 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   comment_text  153164 non-null  object
dtypes: object(1)
memory usage: 2.3+ MB


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159571 entries, 0000997932d777bf to fff46fc426af1f9a
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   comment_text   159571 non-null  object
 1   toxic          159571 non-null  int64 
 2   severe_toxic   159571 non-null  int64 
 3   obscene        159571 non-null  int64 
 4   threat         159571 non-null  int64 
 5   insult         159571 non-null  int64 
 6   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 9.7+ MB


In [7]:
import re

def remove_special_characters(text):
    text = re.sub(r'http\S+', ' ', text )
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\bhttps?://[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)+\b', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\d', ' ', text)  # Corrected line
    text= re.sub(r'[\u4e00-\u9fff]+', ' ', text)
    return text

df_train['comment_text'] = df_train['comment_text'].apply(remove_special_characters)
df_test['comment_text'] = df_test['comment_text'].apply(remove_special_characters)

In [8]:
file_path = '/kaggle/input/toxic-comments-transformer-embeddings/train_embs_400.npy'
df_train_emb = np.load(file_path)

In [9]:
X = df_train_emb  # Assuming 'comment_text' contains your text data
y = df_train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

In [10]:
from sklearn.model_selection import train_test_split
# Splitting the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, random_state=42)

In [11]:
x = X_train
val_x = X_valid

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer


# Initialize MultiLabelBinarizer with the classes
mlb = MultiLabelBinarizer()

# Transform target labels into binary form
y_train_binary = mlb.fit_transform(y_train)
y_valid_binary = mlb.transform(y_valid)

# At this point, y_train_binary and y_valid_binary are ready to be used in your model training


In [13]:
import numpy as np

# List of class labels
class_labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Initialize empty arrays to store binary labels
y_train_binary = np.zeros((len(y_train), len(class_labels)), dtype=int)
y_valid_binary = np.zeros((len(y_valid), len(class_labels)), dtype=int)

# Manually create binary labels for each class
for i, label in enumerate(class_labels):
    y_train_binary[:, i] = y_train[label]
    y_valid_binary[:, i] = y_valid[label]

# At this point, y_train_binary and y_valid_binary are ready to be used in your model training


In [14]:
print("y_train_binary shape:", y_train_binary.shape)
print("y_valid_binary shape:", y_valid_binary.shape)


y_train_binary shape: (127656, 6)
y_valid_binary shape: (31915, 6)


In [15]:
from sklearn.neural_network import MLPClassifier
import numpy as np

epochs = 20
# Define MLPClassifier with custom activation function
clf = MLPClassifier(hidden_layer_sizes=(100, 64, 32, 10, 6),
                    activation='relu',
                    solver='adam',
                    random_state=42,
                    verbose=True,
                    learning_rate_init=0.001,
                    max_iter=epochs,  # Set the maximum number of iterations
                    early_stopping=False  # Disable automatic early stopping
                    )

# Fit the classifier with the training data and binary target labels
clf.fit(X_train, y_train_binary)


Iteration 1, loss = 1.07798265
Iteration 2, loss = 0.38046971
Iteration 3, loss = 0.35263710
Iteration 4, loss = 0.34100986
Iteration 5, loss = 0.33490246
Iteration 6, loss = 0.33101645
Iteration 7, loss = 0.32786738
Iteration 8, loss = 0.32290622
Iteration 9, loss = 0.31997723
Iteration 10, loss = 0.31809351
Iteration 11, loss = 0.31375675
Iteration 12, loss = 0.31180795
Iteration 13, loss = 0.30916600
Iteration 14, loss = 0.30891602
Iteration 15, loss = 0.30513766
Iteration 16, loss = 0.30405045
Iteration 17, loss = 0.30248525
Iteration 18, loss = 0.29992348
Iteration 19, loss = 0.29861748
Iteration 20, loss = 0.29686496


/opt/conda/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(100, 64, 32, 10, 6), max_iter=20,
              random_state=42, verbose=True)

In [16]:
file_path_2 = '/kaggle/input/toxic-comments-transformer-embeddings/test_embs_400.npy'

df_test_emb = np.load(file_path_2)

In [17]:
# Assuming clf is the trained MLPClassifier model

# Predict probabilities for test data
y_test_prob = clf.predict_proba(df_test_emb)

# Print the shape of predicted probabilities for confirmation
print("Shape of predicted probabilities:", y_test_prob.shape)

# Now you have predicted probabilities for each class for the test data


Shape of predicted probabilities: (153164, 6)


In [18]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [19]:
# Convert the predicted probabilities to float
y_test_prob_float = y_test_prob.astype(float)

# Print the updated type of predicted probabilities for confirmation
print("Type of predicted probabilities:", y_test_prob_float.dtype)

# Now y_test_prob_float contains predicted probabilities for each class for each data point in the test data, stored as float numbers


Type of predicted probabilities: float64


In [20]:
import numpy as np

# Assuming y_test_prob_float is your array of predicted probabilities

# Set numpy printing options to suppress scientific notation
np.set_printoptions(suppress=True)

# Print the updated predicted probabilities
print(y_test_prob_float)


[[0.98639619 0.27196872 0.9409973  0.02606734 0.86391813 0.12814601]
 [0.00010814 0.00000001 0.0000421  0.         0.00000971 0.00000011]
 [0.00051815 0.00000015 0.00017551 0.00000003 0.00006705 0.00000204]
 ...
 [0.00041507 0.00000016 0.00016709 0.00000003 0.00005876 0.00000208]
 [0.0006537  0.00000051 0.00030707 0.0000001  0.0001044  0.000005  ]
 [0.96740502 0.02602529 0.87894917 0.00053821 0.60045087 0.00878483]]


In [21]:
# Set the floating-point format for display
pd.options.display.float_format = '{:.10f}'.format

# Create submission DataFrame
submid = pd.DataFrame({'id': df_test.index})  # Use index as 'id' column
submission = pd.concat([submid, pd.DataFrame(y_test_prob_float, columns=classes)], axis=1)

# Write submission DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)


In [22]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.9863961935,0.2719687223,0.9409973025,0.0260673352,0.8639181256,0.1281460077
1,0000247867823ef7,0.0001081359,0.0000000069,0.0000420972,0.0000000006,0.0000097085,0.0000001063
2,00013b17ad220c46,0.0005181513,0.0000001495,0.0001755085,0.0000000312,0.0000670519,0.0000020372
3,00017563c3f7919a,0.0000403473,0.0000000019,0.0000148399,0.0000000002,0.0000038666,0.0000000451
4,00017695ad8997eb,0.0006608668,0.0000001095,0.0001210104,0.0000000492,0.0000827802,0.0000027439
